In [ ]:
## NOTE: you made a mistake implementing EM estimates...double check this 
## looks like generate_EM() is not being run correctly
## might actually be due to only calculating estimates for relevant years and using entire match 
## frame; this might affect the strength

## TO DO: split helper functions into generating elo, then getting 52_stats
## need to make sure that df has all relevant matches (within year range) prior to generating
## EM stats, otherwise the value of 'n' will be thrown off

In [3]:
import pandas as pd
import numpy as np
from data_functions import *
from sklearn.metrics import log_loss, accuracy_score

df = pd.read_csv('match_data_constructed/match_df_6_29_2018')
f_path = '../../repositories/college/research/my_data/elo_atp_matches_all_10_29.csv'
df_old = pd.read_csv(f_path)

In [4]:
1513/3964.

0.3816851664984864

In [5]:
f_t, f_av = .6153, .6468
g_av = 1-f_av
f_i, g_j = .6465, .3478

f_t + (f_i-f_av) - (g_j-g_av)

0.6203999999999998

In [6]:
match_sub = df[(df['p0_name']=='Kevin Anderson') & (df['p1_name']=='Fernando Verdasco')]
match_sub = match_sub[match_sub['match_year']==2013]
# match_sub = match_sub[['p0_name','p1_name','p0_52_rwon','p0_52_rpt','p1_52_rwon','p1_52_rpt']]


In [10]:
df = df[df['tny_name']!='ATP Challenger Tour Finals CH']
df = df[df['tny_name']!='Davis Cup']
df_old = df_old[df_old['tny_name']!='ATP Challenger Tour Finals CH']
df_old = df_old[df_old['tny_name']!='Davis Cup']

YEAR = 2014
sub = df[df['match_year']==YEAR].reset_index()
sub_old = df_old[df_old['match_year']==YEAR].reset_index()

In [14]:
sub.shape

(2488, 75)

In [12]:
a = 0
b = 0
for tny_name in set(sub['tny_name']):
    a += sum(sub['tny_name']==tny_name)
    b += sum(sub_old['tny_name']==tny_name)
    if sum(sub_old['tny_name']==tny_name) != sum(sub['tny_name']==tny_name):
        print tny_name, sum(sub_old['tny_name']==tny_name), sum(sub['tny_name']==tny_name)
print a, b

Chennai 0 21
Doha 0 31
Brisbane 0 27
2488 2409


In [16]:
cols = ['s_pct']
df = generate_EM_stats(sub, cols)

s_pct 0.636546532584


In [147]:
stat_history = np.concatenate([sub['p0_s_pct'],sub['p1_s_pct']],axis=0)
tau2_hat = np.nanvar(stat_history)
print tau2_hat

0.0014281577513


In [148]:
stat_history = np.concatenate([sub_old['p0_s_pct'],sub_old['p1_s_pct']],axis=0)
tau2_hat = np.nanvar(stat_history)
print tau2_hat

0.00143649763282


In [6]:
sub[['tny_name','p0_name','p1_name',\
     'p0_s_kls','p0_s_pct_EM','p1_s_kls','p1_s_pct_EM']].loc[79:84]

,tny_name,p0_name,p1_name,p0_s_kls,p0_s_pct_EM,p1_s_kls,p1_s_pct_EM
79,Auckland,Donald Young,Federico Delbonis,0.573700,0.618342,0.589324,0.641060
80,Auckland,Santiago Giraldo,Albert Montanes,0.645451,0.624250,0.660674,0.620087
81,Auckland,Guillermo Garcia,Robin Haase,0.638478,0.612437,0.634435,0.631945
82,Auckland,Marcos Baghdatis,Steve Johnson,0.733536,0.639615,0.692084,0.635690
83,Auckland,Igor Sijsling,Yen Hsun,0.655477,0.648859,0.667468,0.629183
84,Auckland,Bradley Klahn,Daniel Brands,0.746740,0.656862,0.712967,0.653501


In [18]:
sub_old[['tny_name','p0_name','p1_name',\
         'p0_s_kls','p0_s_pct_JS','p1_s_kls','p1_s_pct_JS']].loc[:5]

,tny_name,p0_name,p1_name,p0_s_kls,p0_s_pct_JS,p1_s_kls,p1_s_pct_JS
0,Auckland,Donald Young,Federico Delbonis,0.573700,0.574336,0.589324,0.639250
1,Auckland,Albert Montanes,Santiago Giraldo,0.660674,0.613221,0.645451,0.618632
2,Auckland,Guillermo Garcia,Robin Haase,0.638478,0.604832,0.634435,0.629601
3,Auckland,Steve Johnson,Marcos Baghdatis,0.692084,0.629776,0.733536,0.637896
4,Auckland,Yen Hsun,Igor Sijsling,0.667468,0.624290,0.655477,0.649739
5,Auckland,Bradley Klahn,Daniel Brands,0.746740,0.669786,0.712967,0.655438


In [ ]:
stat_history = np.concatenate([df['p0_s_kls'],df['p1_'+col]],axis=0)

In [111]:
prob_cols = [u'match_prob_kls', u'match_prob_kls_EM', u'match_prob_adj_kls', \
             u'match_prob_adj_kls_EM', u'elo_prob']
for col in prob_cols:
    print col
    print 'accuracy score: ', accuracy_score(1 - sub['winner'], sub[col] >=.5)
    print 'log loss: ',log_loss(1 - sub['winner'], sub[col])

match_prob_kls
accuracy score:  0.648713826367
log loss:  0.647957708155
match_prob_kls_EM
accuracy score:  0.646704180064
log loss:  0.613318223449
match_prob_adj_kls
accuracy score:  0.674839228296
log loss:  0.631435350174
match_prob_adj_kls_EM
accuracy score:  0.674839228296
log loss:  0.621415954217
elo_prob
accuracy score:  0.690514469453
log loss:  0.588863185483


In [110]:
prob_cols = [u'match_prob_kls', u'match_prob_kls_JS', u'match_prob_adj_kls', \
             u'match_prob_adj_kls_JS', u'elo_prob']
for col in prob_cols:
    print col
    print 'accuracy score: ', accuracy_score(sub_old['winner'], sub_old[col] >=.5)
    print 'log loss: ',log_loss(sub_old['winner'], sub_old[col])

match_prob_kls
accuracy score:  0.64798671648
log loss:  0.649029598611
match_prob_kls_JS
accuracy score:  0.654628476546
log loss:  0.615573317508
match_prob_adj_kls
accuracy score:  0.678704856787
log loss:  0.631634020959
match_prob_adj_kls_JS
accuracy score:  0.679119966791
log loss:  0.616994785515
elo_prob
accuracy score:  0.690743046907
log loss:  0.585792277895


In [57]:
sub[['match_year','tny_name','p0_name','p1_name','p0_52_swon','p0_52_svpt',\
     'p1_52_swon','p1_52_svpt']].loc[7]

match_year                 2013
tny_name        Australian Open
p0_name          Kevin Anderson
p1_name       Fernando Verdasco
p0_52_swon                 3191
p0_52_svpt                 4695
p1_52_swon                 2469
p1_52_svpt                 3819
Name: 7, dtype: object

In [58]:
sub_old[['match_year','tny_name','p0_name','p1_name','p0_52_swon','p0_52_svpt',\
     'p1_52_swon','p1_52_svpt']].loc[7]

match_year                 2013
tny_name        Australian Open
p0_name          Kevin Anderson
p1_name       Fernando Verdasco
p0_52_swon                 3191
p0_52_svpt                 4695
p1_52_swon                 2469
p1_52_svpt                 3819
Name: 7, dtype: object

In [ ]:
# YEAR = 2013
# sub = df[(df['p0_name']=='Kevin Anderson') | (df['p1_name']=='Kevin Anderson')]
# sub_old = df_old[(df_old['p0_name']=='Kevin Anderson') | (df_old['p1_name']=='Kevin Anderson')]
# sub = sub[sub['match_year']==YEAR].reset_index()
# sub_old = sub_old[sub_old['match_year']==YEAR].reset_index()